## Tutorials for training Non-SA-MIL

In [ ]:
####################
###  LIBRARIES  ####
####################

import numpy as np
import warnings
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

# Remove TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import TensorFlow and Keras for neural network operations
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Loss
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

# Set the default float type for TensorFlow to "float32"
tf.keras.backend.set_floatx("float32")

# Print the number of available GPUs
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
####################
### DATA LOADING ###
####################

print('Starting preprocessing of bags')

# Define directories for image files, have training images from three folders
train_images_dir1 = './Data/train1/'
train_images_dir2 = './Data/train2/'
train_images_dir3 = './Data/train3/'

# Get lists of files in the directories
train_files1 = set(os.listdir(train_images_dir1))
train_files2 = set(os.listdir(train_images_dir2))
train_files3 = set(os.listdir(train_images_dir3))

# Read bag data from CSV files
train_bags = pd.read_csv("./tables/Training_examples.csv")

# Create a mapping of train files to their respective directories
dirs_ = [train_images_dir1, train_images_dir2, train_images_dir3]
train_files_loc = {
    k: dirs_[
        (k[:-4]+'.npy' in train_files1) * 1 +
        (k[:-4]+'.npy' in train_files2) * 2 +
        (k[:-4]+'.npy' in train_files3) * 3 - 1
    ]
    for k in train_bags.instance_name
}

# Create lists of DCM files for train files in each directory
train_files1_dcm = [k[:-4] + '.dcm' for k in train_files1]
train_files2_dcm = [k[:-4] + '.dcm' for k in train_files2]
train_files3_dcm = [k[:-4] + '.dcm' for k in train_files3]

# Filter train bags based on DCM file existence
train_bags = train_bags[
    train_bags.instance_name.isin(train_files1_dcm) |
    train_bags.instance_name.isin(train_files2_dcm) |
    train_bags.instance_name.isin(train_files3_dcm)
]

In [ ]:
##########################
### BAGS PREPROCESSING ###
##########################

# Set the desired bag size
bag_size = 57

# Create additional train bags to reach the desired bag size
added_train_bags = pd.DataFrame()
for idx in train_bags.bag_name.unique():
    bags = train_bags[train_bags.bag_name==idx].copy()
    num_add = bag_size - len(bags.instance_name)

    aux = bags.iloc[0].copy()
    aux.instance_label = 0
    aux.instance_name = 'all_zeros'
    for i in range(num_add):
        added_train_bags = added_train_bags.append(aux)

train_bags = train_bags.append(added_train_bags)

# Convert bags data to dictionaries for optimization
train_bags_dic = {k: list(train_bags[train_bags.bag_name==k].instance_name) for k in train_bags.bag_name.unique()}

In [ ]:
####################
###  DATALOADER  ###
####################

dim=(512,512,bag_size)
class DataGeneratorMIL(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, labels=None, batch_size=256, dim=(512,512,512), n_channels=3,
                 n_classes=2, shuffle=True, is_train=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.is_train = (labels is not None) and is_train
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        list_IDs_temp = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]

        X = self.__data_generation(list_IDs_temp)
        # Generate data
        if self.is_train:
            y = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            return np.array(X), np.array(y, dtype='float64')
        else:
            return np.array(X)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            if self.is_train:
                ids = train_bags_dic[ID]
            else:
                ids = test_bags_dic[ID]
            imgs = []
            for idx in ids:
                if idx == 'all_zeros':
                    img = np.zeros((self.dim[0], self.dim[1], self.n_channels))
                    imgs.append(img)
                    continue
                if self.is_train:
                    _dir = train_files_loc[idx]
                    img = np.load(_dir + idx[:-4] + '.npy')
                    img = cv2.resize(img, (self.dim[1], self.dim[0]))
                    imgs.append(img)
                else:
                    img = np.load(test_images_dir + idx[:-4] + '.npy')
                    img = cv2.resize(img, (self.dim[1], self.dim[0]))
                    imgs.append(img)
            X[i,] = np.transpose(imgs, [1,2,0,3])

        return X

In [ ]:
########################
### TRAIN/TEST SPLIT ###
########################

from sklearn.model_selection import train_test_split

N = len(train_bags.bag_name.unique())
bags = train_bags.groupby('bag_name').max()

X_train, X_val, y_train, y_val = train_test_split(np.array(bags.index)[:], bags.bag_label[:],
                                                 test_size=0.20, random_state=0,
                                                 stratify=bags.bag_label[:])

batch_size = 4

# Creating the train dataset using DataGeneratorMIL
train_dataset = DataGeneratorMIL(X_train, y_train, batch_size=batch_size, dim=dim)

# Creating the validation dataset using DataGeneratorMIL
val_dataset = DataGeneratorMIL(X_val, y_val, batch_size=batch_size, dim=dim, is_augment=False)

# Calculating class weights for imbalanced classes
classes, counts = np.unique(train_dataset.labels, return_counts=True)
class_weights = {k: (1 - v/sum(counts)) for k, v in zip(classes, counts)}


In [ ]:
####################
###    MODEL     ###
####################

# MILAttentionLayer
class MILAttentionLayer(layers.Layer):
    """Implementation of the attention-based Deep MIL layer."""

    def __init__(
        self,
        weight_params_dim,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        use_gated=False,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.weight_params_dim = weight_params_dim
        self.use_gated = use_gated

        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

        self.v_init = self.kernel_initializer
        self.w_init = self.kernel_initializer
        self.u_init = self.kernel_initializer

        self.v_regularizer = self.kernel_regularizer
        self.w_regularizer = self.kernel_regularizer
        self.u_regularizer = self.kernel_regularizer

    def build(self, input_shape):
        input_dim = input_shape[1]

        self.v_weight_params = self.add_weight(
            shape=(input_dim, self.weight_params_dim),
            initializer=self.v_init,
            name="v",
            regularizer=self.v_regularizer,
            trainable=True,
        )

        self.w_weight_params = self.add_weight(
            shape=(self.weight_params_dim, 1),
            initializer=self.w_init,
            name="w",
            regularizer=self.w_regularizer,
            trainable=True,
        )

        if self.use_gated:
            self.u_weight_params = self.add_weight(
                shape=(input_dim, self.weight_params_dim),
                initializer=self.u_init,
                name="u",
                regularizer=self.u_regularizer,
                trainable=True,
            )
        else:
            self.u_weight_params = None

        self.input_built = True

    def call(self, inputs):
        instances = self.compute_attention_scores(inputs)
        instances = tf.reshape(instances, shape=(-1, dim[2]))
        alpha = tf.math.softmax(instances, axis=1)
        return alpha

    def compute_attention_scores(self, instance):
        original_instance = instance
        instance = tf.math.tanh(tf.tensordot(instance, self.v_weight_params, axes=1))

        if self.use_gated:
            instance = instance * tf.math.sigmoid(
                tf.tensordot(original_instance, self.u_weight_params, axes=1)
            )

        return tf.tensordot(instance, self.w_weight_params, axes=1)


# Model
num_data = batch_size
D = bag_size

Conv1 = layers.Conv2D(16, (5, 5), data_format="channels_last", activation='relu', kernel_initializer='glorot_uniform', padding='same')
Conv2 = layers.Conv2D(32, (3,3),  data_format="channels_last", activation='relu')
Conv3 = layers.Conv2D(32, (3,3),  data_format="channels_last", activation='relu')
Conv4 = layers.Conv2D(32, (3,3),  data_format="channels_last", activation='relu')
Conv5 = layers.Conv2D(32, (3,3),  data_format="channels_last", activation='relu')
Conv6 = layers.Conv2D(32, (3,3),  data_format="channels_last", activation='relu')

def VGG(inp):
    inp = tf.reshape(tf.transpose(inp, perm=(0,3,1,2,4)), shape=(-1, dim[0], dim[1], 3))
    x = Conv1(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), data_format="channels_last", strides=(2, 2))(x)
    x = Conv2(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), strides=(2, 2), data_format="channels_last")(x)
    x = layers.Dropout(0.3)(x)

    x = Conv3(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), strides=(2, 2), data_format="channels_last")(x)
    x = Conv4(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), strides=(2, 2), data_format="channels_last")(x)

    x = Conv5(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), strides=(2, 2), data_format="channels_last")(x)
    x = layers.Dropout(0.3)(x)

    x = Conv6(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPool2D((2, 2), strides=(2, 2), data_format="channels_last")(x)
    x = layers.Dropout(0.3)(x)

    return layers.Flatten()(x)

def build_model():
    inp = keras.Input(shape=(*dim, 3))
    H = VGG(inp)
    A = MILAttentionLayer(
        weight_params_dim=64,
        kernel_regularizer=keras.regularizers.l2(0.01),
        use_gated=True,
        name="alpha",
    )(H)
    H = tf.reshape(H, shape=(-1, dim[2], H.shape[1]))
    A = tf.expand_dims(A, axis=1)
    intermediate = tf.linalg.matmul(A, H)
    intermediate = tf.squeeze(intermediate, axis=1)
    intermediate = layers.Dropout(0.25)(intermediate)
    intermediate = layers.Dense(128)(intermediate)
    out = layers.Dense(1, activation='sigmoid')(intermediate)
    return keras.Model(inputs=inp, outputs=out)

model = build_model()

auc = tf.keras.metrics.AUC()
adam = tf.compat.v1.train.AdamOptimizer(learning_rate=5e-5)
model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics=[auc, 'accuracy']
)
earlyStopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='min')
print(model.summary())

In [ ]:
####################
###    Train   ###
####################
for i in range(0, 5):
    checkpoint_path = "./model/att_{}.ckpt".format(i)
    checkpoint_dir = os.path.dirname(checkpoint_path)

    cp_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        verbose=1,
        mode='min'
    )


    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=200,
        callbacks=[earlyStopping, cp_callback],
        class_weight=class_weights
    )

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = './log.csv'

    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)